# Pet Names (Python)

This trains a neural network with Keras that generates pet names. It saves the neural to a file at the end, to use in a function in a separate notebook.

First, load up the appropriate TensorFlow and other libraries.

In [ ]:
import numpy as np
import io
import itertools
import pandas as pd
import random
import string
from tensorflow import keras
from tensorflow.keras import layers

Next, create some lookups and variables. The character list is the set of characters to use in the names. The lookup is that converted into a dictionary of integers for encoding.

In [ ]:
character_list = list(string.ascii_lowercase) + [".","-"," ","+"]
character_lookup = dict(zip(character_list, range(len(character_list))))
max_length = 10
num_characters = len(character_lookup)

This loads the data, converts it to lower case, and removes any pet that doesn't have a name or species (or the name has invalid characters).

In [ ]:
def load_pet_data():
    """Load the pet data
    This loads the pet data from the csv file and cleans it appropriately.
    It removes extra columns, rows with malformed names or breeds, and makes the names
    lowercase.
    """
    pet_data = (pd.read_csv("seattle_pet_licenses.csv", dtype = {"Animal's Name": str, 'Species': str,'Primary Breed': str,
                                                        'Secondary Breed': str},
                                                        usecols=["Animal's Name",'Species',
                                                        'Primary Breed','Secondary Breed'])
                    .rename(columns = {"Animal's Name":"name",
                     "Species": "species",
                     "Primary Breed": "primary_breed",
                     "Secondary Breed": "secondary_breed"})
                    .dropna(subset=['name', 'species']))

    for column in pet_data.columns:
        pet_data[column] = pet_data[column].str.lower()

    pet_data = pet_data[pet_data["name"].str.match("^[ \\.a-z-]+$")]

    return pet_data

pet_data = load_pet_data()

This code below:

1. Has a function that converts the DataFrame into a list of objects, where each object has some metadata plus the list of subsequences (partials of the name string. So like "spot", has ["s", "sp", "spo", "spot", "spot+"])
2. Grabs the subsequences, which then are a list of lists, and flattens them into a single list.
3. Stores that list as a new column in the data frame

In [ ]:
def make_subsequences(name):
    characters = name + '+'
    subsequences = [list(characters[0:(i+1)]) for i in range(len(characters))]
    return subsequences

def get_all_subsequences(pet_data):
    subsequences = pet_data["name"].map(make_subsequences)
    return subsequences

pet_data['subsequences'] = get_all_subsequences(pet_data)

Then we pull the subsequences, shuffle them, convert the characters to ints using the lookup, pad them, and one hot encode them. You could also use the other columns such as the breed in the neural network if you so chose.

In [ ]:
def characters_to_matrix(character_data):
    character_data = [[character_lookup[chr] for chr in c] for c in character_data]
    padded_character_data = keras.utils.pad_sequences(character_data, maxlen = max_length+1)
    text_matrix = keras.utils.to_categorical(padded_character_data, num_classes = num_characters)
    return text_matrix

expanded_data = pet_data.explode("subsequences")
character_data = expanded_data["subsequences"].tolist()
random.shuffle(character_data)
text_matrix = characters_to_matrix(character_data)

x_name = text_matrix[:,range(max_length),:]
y_name = text_matrix[:,max_length,:]

Now we start the neural network part. Below is the model architecture definition.

In [ ]:
model = keras.Sequential(
    [
        keras.Input(shape=(max_length, num_characters)),
        layers.LSTM(32, return_sequences = True),
        layers.LSTM(32),
        layers.Dropout(0.2),
        layers.Dense(num_characters, activation="softmax"),
    ]
)
optimizer = keras.optimizers.RMSprop(learning_rate=0.01)
model.compile(loss="categorical_crossentropy", optimizer=optimizer)

This is training the model.

In [ ]:
model.fit(x_name, y_name, batch_size = 64, epochs = 12)

Last we save the model.

In [ ]:
model.save("model.h5")